<a href="https://colab.research.google.com/github/shininglight99/ellison-archive/blob/master/Yieldcurve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as optimize
from datetime import datetime

#utils

def geo_sum(Iam, n):
  return Iam *  (1-Iam**n) / (1-Iam)


#vanila bond

class ZCB:
  def __init__(self,F,T,m=2):
    self.F=F
    self.T=T
    self.m=m
    self.n=self.m*self.T

  def price(self,discount_factor = None, ytm = None, continuous = None):
    if ytm:
      if continuous:
        Iam = np.exp(-ytm/self.m)
      else:
        Iam = 1/(1 + ytm / self.m)
      return self.F *Iam **self.n
    
    if self.T not in discount_factor:
      raise Exception(f"Discount factor at {self.T} must be given")
    return self.F * discount_factor[self.T]

  def ytm(self, price, continuous= False , guess=0.05):
    func = lambda ytm: self.price(ytm=ytm , continuous = continuous) -price
    return optimize.newton(func, guess)

  def dv01(self, ytm):
    if not ytm:
      raise Exception(f"YTM must be given")
      return self.F * self.T *(1+ytm/sef.m)**(self.n + 1) / 10000

  def duration(self, ytm):
    return self.T / (1 + ytm / self.m)

  def mac_duration(self, ytm):
    return self.T
    
  def convextity(self, ytm):
    return self.T *(self.T + 1/self.m)/(1+ytm / self.m)**(self.m)



  

In [3]:
class CouponBond:
  def __init__(self, F, T, q, m=2):
    self.F = F
    self.T = T
    self.q = q
    self.m = m
    self.n = self.m *self.T
    self.coupon = self.F *self.q / self.m

  def price(self, discount_factor = None, ytm = None, continuous = False):
    if ytm:
      if continuous:
        Iam = np.exp(-ytm/ self.m)
      else:
        Iam = 1 / (1+ytm / self.m)

    return self.F *Iam **self.n + self.coupon * geo_sum(Iam, self.n)

    dt = 1/ self.m
    years = np.arrange(dt, self.T + dt, dt)
    if not set(years).issubset(discount_factor.index):
        raise Exception(f"Missing years in discount_factor")
    return self.F * discount_factor[self.T] + self.coupon * discount_factor[years].sum()

  def between_coupon(self, reference_date, maturity):
    maturity = datetime.strptime(maturity, "%Y-%m-%d")
    reference_date = datetime.strptime(reference_date, "%Y-%m-%d")
    time_delta = (maturity - reference_date)
    k = int(time_delta.days / (365 / self.m) + 1 )
    tau= time_delta.days / (365/ self.m) - (k-1)
    return k, tau
  def full_price(self, ytm, k, tau, continuous= False):
    if ytm:
      if continuous:
        Iam = np.exp(-ytm/ self.m)
      else:
        Iam = 1/ (1+ytm/ self.m)
    Iam = 1 / (1+ ytm/self.m)
    return self.F * Iam**(k+tau) + self.coupon * Iam **tau *geo_sum(Iam , k)

  def ytm(self, price, k= None , tau = None, continuous = False, guess = 0.05, between_coupon = False):
    if between_coupon:
      func = lambda ytm : self.full_price(ytm = ytm, k=k, tau=tau , continuous = continuous)-price
    else:
      func = lambda ytm : self.price(ytm= ytm, continuous = continuous)-price
    return optimize.newton(func, guess)  


MarketWatch US Treasury Bond

In [4]:
F= 100
m = 2
q= 0.00875
T = 10

#coupon bond
bond = CouponBond (T= T, q = q, F = F, m = m)
price = 98 +4/32
print("YTM: ", bond.ytm(price=price))


YTM:  0.010732437559411013


In [5]:
F = 100
m = 2
q = 0.00875

#coupon bond
bond= CouponBond(T=T, q=q, F=F, m=m)
price = 98 + 4/32

reference_date = datetime.today().strftime("%Y-%m-%d")
maturity_date = "2030-11-15"

k, tau = bond.between_coupon(reference_date, maturity_date)

print("YTM Between Coupon Payment : ", \
      bond.ytm(price=price, k=k, tau=tau, between_coupon= True))


YTM Between Coupon Payment :  0.010417683984502912


In [10]:
F = 100
m = 2
q = 0.00125
T = 2


#coupon bond
bond = CouponBond(T=T, q=q, F=F, m=m)
price = 100 + 0/32

reference_date = datetime. today().strftime("%Y-%m-%d")
maturity_date = "2030-11-15"

k,tau = bond.between_coupon(reference_date, maturity_date)

print("YTM Between Coupon Payment: ",\
      bond.ytm(price=price, k=k, tau=tau, between_coupon = True))

print("YTM: ", bond.ytm(price=price))

YTM Between Coupon Payment:  0.001214671215017434
YTM:  0.001250000000000043


In [9]:
F= 100
m = 2
q= 0.00125
T = 3

#coupon bond
bond = CouponBond (T=T, q=q, F=F, m=m)
price = 99 +8/32

reference_date = datetime.today().strptime("%Y-%m-%d")


TypeError: ignored